In [1]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import cv2
import easyocr

from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.widgets import  Slider

In [2]:
VIDEO_PATH = "Videos/Start1.avi"

In [3]:
CAP = cv2.VideoCapture(VIDEO_PATH)
FPS = CAP.get(cv2.CAP_PROP_FPS)
LENTH = int(CAP.get(cv2.CAP_PROP_FRAME_COUNT) / FPS)


In [4]:
%matplotlib qt
# Preprocess
def image_process(img):
    return img


fig, ax = plt.subplots()
fig.subplots_adjust(left=0, right=1, top=1, hspace=0, bottom=0.2)
# del ax.xaxis, ax.yaxis

axfreq = fig.add_axes([0.2, 0.1, 0.6, 0.05])
TIME_slider = Slider(
    ax=axfreq,
    label='Time',
    valmin=0,
    valmax=LENTH,
    valinit=0,
    valstep=1,
)

def update(val):
    time = TIME_slider.val
    CAP.set(cv2.CAP_PROP_POS_FRAMES, int(FPS*time))
    _, frame = CAP.read()

    
    plot.set_data(frame)
    plot.autoscale()

    fig.canvas.draw_idle()
TIME_slider.on_changed(update)


CAP.set(cv2.CAP_PROP_POS_FRAMES, 0)
_, frame = CAP.read()
plot = ax.imshow(frame,cmap='binary')

# Recognize

In [ ]:
cap = cv2.VideoCapture("Videos/Full_font2.avi")

In [ ]:
## Saving
# fps =int(cap.get(cv2.CAP_PROP_FPS))
# diap = tqdm(iterable=range(0,fps*40,fps))
# for i in diap:
#     cap.set(cv2.CAP_PROP_POS_FRAMES, i)
#     ret, frame = cap.read()
#     if ret:
#         cv2.imwrite(f"Images/frame{int(i/fps)}.jpg",frame)

In [ ]:
def recognize(image, thresh):
    frame_2color = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh, frame_bw = cv2.threshold(frame_2color, thresh, 255,
                                     cv2.THRESH_BINARY)
    return tes.image_to_string(frame_bw), frame_bw


In [ ]:
# Check
THRESH = 147

cap = cv2.VideoCapture("Videos/Full_font2.avi")
pdf_file = PdfPages('Full_font2.pdf')

fps = int(cap.get(cv2.CAP_PROP_FPS))
diap = tqdm(iterable=range(0, fps * 40, fps))

for i in diap:
    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
    ready, frame = cap.read()
    text, frame_treated = recognize(frame, thresh=THRESH)
    text = text.replace('\n', '')
    text = f"t={i/fps: .1f}|{text}"
    if ready:
        fig, (ax_row, ax_treated) = plt.subplots(ncols=2)

        ax_row.imshow(frame)
        ax_treated.imshow(frame_treated)
        ax_row.text(0, 0, text, fontsize=15)

        fig.savefig(pdf_file, format='pdf')
        plt.close()
pdf_file.close()

# One frame tests

In [ ]:
cap = cv2.VideoCapture()
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
ready, frame = cap.read()
frame = frame
# [200:280]
plt.imshow(frame)

In [ ]:
frame2 = frame[:, :, :]
frame2 = (frame2 + 1) % 256
plt.imshow(frame2)

In [ ]:
reader = easyocr.Reader(['en'])

In [ ]:
result = reader.readtext(frame)

# test easy ocr

In [ ]:
cap = cv2.VideoCapture("Videos/Start5.avi")
pdf_file = PdfPages('Easy_Start5_test.pdf')

fps = int(cap.get(cv2.CAP_PROP_FPS))
video_len = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) / fps)

diap = tqdm(iterable=range(fps * 87, fps * 89, int(fps / 5)))
try:
    for i in diap:
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ready, frame = cap.read()
        frame = frame[100:142]
        result = reader.readtext(frame)

        if ready:
            fig, ax = plt.subplots()

            ax.imshow(frame)
            text_result = ' -- '.join([
                f'{value}({confidence*100:.0f})'
                for _, value, confidence in result
            ])
            ax.set_title(f'T= {round(i/fps,1)}  R= {text_result}')

            fig.savefig(pdf_file, format='pdf')
            plt.close()
except:
    print('Error has been detected')
finally:
    pdf_file.close()

In [ ]:
' '.join([value for _, value, confidence in result])


# cut picture

In [ ]:
import plotly.express as px
from skimage import data

img = data.chelsea()
fig = px.imshow(img)
# fig.add_annotation(
#     x=0.5,
#     y=0.9,
#     text="Drag and draw annotations",
#     xref="paper",
#     yref="paper",
#     showarrow=False,
#     font_size=20, font_color='cyan')
# Shape defined programatically
# fig.add_shape(
#     type='rect',
#     x0=230, x1=290, y0=230, y1=280,
#     xref='x', yref='y',
#     line_color='cyan'
# )
# Define dragmode, newshape parameters, amd add modebar buttons


# fig.update_layout(dragmode='drawrect', newshape=dict(line_color='cyan'))

fig.show(config={'modeBarButtonsToAdd': ['drawrect', 'eraseshape']})


In [ ]:
import plotly.graph_objs as go
from plotly.colors import n_colors

# Загрузка изображения
img = go.Image(z=frame)

# Определение области, которую нужно обрезать
clip_path = go.Scatter(
    x=[100, 300, 300, 100],
    y=[100, 100, 300, 300],
    fill='toself'
)

# Определение фигуры с изображением и обрезанной областью
fig = go.Figure(data=[img])
fig.update_layout(width=600, height=600)
fig.update_layout(images=[dict(
    data=img,
    xref='x',
    yref='y',
    x=clip_path.x[0],
    y=clip_path.y[0],
    sizex=clip_path.x[2] - clip_path.x[0],
    sizey=clip_path.y[2] - clip_path.y[0],
    sizing='stretch',
    opacity=1.0,
    layer='below',
    # sourcetype='image',
    # sourceopacity=1.0,
    # clippath=f"test.jpg",
    # cliponaxis=False
)])

# Определение цветов для обрезки
colors = n_colors('rgb(255, 0, 0)', 'rgb(0, 0, 255)', 10)

# Определение состояния для обрезки
state = {'color': 0}

# Обработчик событий для кликов мыши
def handle_click(trace, points, state):
    if len(points.point_inds) > 0:
        state['color'] = (state['color'] + 1) % len(colors)
        color = colors[state['color']]
        fig.update_traces(patch=dict(fillcolor=color))

# Добавление обработчика событий для кликов мыши
fig.data[0].on_click(lambda trace, points, state: handle_click(trace, points, state))

# Отображение фигуры
fig.show()